In [2]:
import sys
#stdout = sys.stdout
#reload(sys)
#sys.setdefaultencoding('utf-8')
#sys.stdout = stdout

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import math
import json

# pd.options.mode.chained_assignment = None

from astropy.table import Table, Column
#from string import join
import re,pickle
from sklearn.preprocessing import label_binarize, OneHotEncoder, LabelEncoder, MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn import metrics
# imports for sklearn transformers
from sklearn.metrics import accuracy_score, precision_recall_curve, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

# imports for sklearn preprocessing transformers
from sklearn.preprocessing import *

# Feature extraction packages
from sklearn.feature_extraction import *
from sklearn.feature_extraction.text import *
from collections import defaultdict
from sklearn.pipeline import Pipeline, FeatureUnion

import os
#os.environ["ML_SDK_CONF_BUCKET"]="stage-ml-platform-sdk"
#from mlsdk.MLApi import MLApi
# import seaborn as sns
# sns.set_style("dark")
# plt.rcParams['figure.figsize'] = 16, 12

In [3]:
pd.set_option('display.max_columns', 500)

# Data Loading

In [4]:
df_raw_feat_and_label = pd.read_csv('train_sample.csv', delimiter = ',', na_values = ['nan', 'NA', 'null', 'NULL'])

In [5]:
df_raw_feat_and_label.shape

(100000, 8)

In [6]:
df_raw_feat_and_label.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [7]:
print(df_raw_feat_and_label['is_attributed'].value_counts())


0    99773
1      227
Name: is_attributed, dtype: int64


In [8]:
df_raw_feat_and_label.describe(include='all')

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000,227,100000.000000
unique,NaN,NaN,NaN,NaN,NaN,80350,227,NaN
top,NaN,NaN,NaN,NaN,NaN,2017-11-08 12:01:02,2017-11-07 23:11:40,NaN
freq,NaN,NaN,NaN,NaN,NaN,7,1,NaN
mean,91255.879670,12.04788,21.771250,22.818280,268.832460,NaN,NaN,0.002270
std,69835.553661,14.94150,259.667767,55.943136,129.724248,NaN,NaN,0.047591
min,9.000000,1.00000,0.000000,0.000000,3.000000,NaN,NaN,0.000000
25%,40552.000000,3.00000,1.000000,13.000000,145.000000,NaN,NaN,0.000000
50%,79827.000000,12.00000,1.000000,18.000000,258.000000,NaN,NaN,0.000000
75%,118252.000000,15.00000,1.000000,19.000000,379.000000,NaN,NaN,0.000000


In [9]:
df_raw_feat_and_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
ip                 100000 non-null int64
app                100000 non-null int64
device             100000 non-null int64
os                 100000 non-null int64
channel            100000 non-null int64
click_time         100000 non-null object
attributed_time    227 non-null object
is_attributed      100000 non-null int64
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


In [10]:
cat=['ip','app','device','os','channel','is_attributed']
for feat in cat:
    df_raw_feat_and_label[feat]=df_raw_feat_and_label[feat].astype('category')
    
df_raw_feat_and_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
ip                 100000 non-null category
app                100000 non-null category
device             100000 non-null category
os                 100000 non-null category
channel            100000 non-null category
click_time         100000 non-null object
attributed_time    227 non-null object
is_attributed      100000 non-null category
dtypes: category(6), object(2)
memory usage: 4.2+ MB


In [11]:
df_raw_feat_and_label.describe(include='all')

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
count,100000,100000,100000,100000,100000,100000,227,100000
unique,34857,161,100,130,161,80350,227,2
top,5348,3,1,19,280,2017-11-08 12:01:02,2017-11-07 23:11:40,0
freq,669,18279,94338,23870,8114,7,1,99773


In [12]:
df_raw_feat_and_label['click_time']=pd.to_datetime(df_raw_feat_and_label['click_time'])
df_raw_feat_and_label.info()
df_raw_feat_and_label.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
ip                 100000 non-null category
app                100000 non-null category
device             100000 non-null category
os                 100000 non-null category
channel            100000 non-null category
click_time         100000 non-null datetime64[ns]
attributed_time    227 non-null object
is_attributed      100000 non-null category
dtypes: category(6), datetime64[ns](1), object(1)
memory usage: 4.2+ MB


,ip,app,device,os,channel,click_time,attributed_time,is_attributed
count,100000.0,100000.0,100000.0,100000.0,100000.0,100000,227,100000.0
unique,34857.0,161.0,100.0,130.0,161.0,80350,227,2.0
top,5348.0,3.0,1.0,19.0,280.0,2017-11-08 12:01:02,2017-11-07 23:11:40,0.0
freq,669.0,18279.0,94338.0,23870.0,8114.0,7,1,99773.0
first,NaN,NaN,NaN,NaN,NaN,2017-11-06 16:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2017-11-09 15:59:51,NaN,NaN


In [13]:
#get weekday part of day from click time
import time
import datetime as dt
from datetime import date
#df_raw_feat_and_label['click_hour']=df_raw_feat_and_label['click_time'].dt.hour 
#df_raw_feat_and_label['click_time'].dt.weekday
labels=[1,2,3,4,5,6]
df_raw_feat_and_label['click_part_of_day']=pd.cut(df_raw_feat_and_label['click_time'].dt.hour , bins=6, labels=labels)
df_raw_feat_and_label['click_is_weekend']=df_raw_feat_and_label['click_time'].dt.weekday.map({0:0,1:0,2:0,3:0,4:0,5:1,6:1})

In [14]:
df_raw_feat_and_label.describe(include='all')
df_raw_feat_and_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
ip                   100000 non-null category
app                  100000 non-null category
device               100000 non-null category
os                   100000 non-null category
channel              100000 non-null category
click_time           100000 non-null datetime64[ns]
attributed_time      227 non-null object
is_attributed        100000 non-null category
click_part_of_day    100000 non-null category
click_is_weekend     100000 non-null int64
dtypes: category(7), datetime64[ns](1), int64(1), object(1)
memory usage: 5.1+ MB


In [15]:
df_raw_feat_and_label['click_is_weekend']=df_raw_feat_and_label['click_is_weekend'].astype('category')
df_raw_feat_and_label.describe(include='all')

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_part_of_day,click_is_weekend
count,100000.0,100000.0,100000.0,100000.0,100000.0,100000,227,100000.0,100000.0,100000.0
unique,34857.0,161.0,100.0,130.0,161.0,80350,227,2.0,6.0,1.0
top,5348.0,3.0,1.0,19.0,280.0,2017-11-08 12:01:02,2017-11-07 23:11:40,0.0,2.0,0.0
freq,669.0,18279.0,94338.0,23870.0,8114.0,7,1,99773.0,21607.0,100000.0
first,NaN,NaN,NaN,NaN,NaN,2017-11-06 16:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2017-11-09 15:59:51,NaN,NaN,NaN,NaN


In [16]:
df_raw_feat_and_label['click_time'].dt.date.unique()
# is weekend is of no use

array([datetime.date(2017, 11, 7), datetime.date(2017, 11, 9),
       datetime.date(2017, 11, 8), datetime.date(2017, 11, 6)], dtype=object)

In [35]:
pd.crosstab(df_raw_feat_and_label.app,df_raw_feat_and_label.is_attributed)

KeyError: -1

In [54]:
pd.crosstab(df_raw_feat_and_label.device,df_raw_feat_and_label.is_attributed)

is_attributed,0,1
device,,
0,489,52
1,94192,146
2,4343,2
4,2,1
5,7,0
6,7,1
7,2,0
9,1,0
11,1,0


In [18]:
pd.crosstab(df_raw_feat_and_label.channel,df_raw_feat_and_label.is_attributed)

is_attributed,0,1
channel,,
3,486,2
4,1,0
5,2,5
13,67,0
15,2,0
17,115,0
18,25,0
19,678,0
21,161,19


In [33]:
df_raw_feat_and_label.groupby(['app','is_attributed']).size().reset_index(name='counts')[(df_raw_feat_and_label.groupby(['app','is_attributed']).size().reset_index(name='counts')['counts']==0) & (df_raw_feat_and_label.groupby(['app','is_attributed']).size().reset_index(name='counts')['is_attributed']==1)]

,app,is_attributed,counts


In [34]:
df_raw_feat_and_label.groupby(['app','is_attributed']).size().reset_index(name='counts')

,app,is_attributed,counts
0,1,0,3135
1,2,0,11737
2,3,0,18275
3,3,1,4
4,4,0,58
5,5,0,174
6,5,1,14
7,6,0,1303
8,7,0,981
9,8,0,2000
